In [1]:
import pandas as pd
import numpy as np
pd.options.display.float_format = '{:20,.4f}'.format

## Bring in the Data

In [2]:
test = pd.read_csv("test.csv")
del test["GarageYrBlt"]
del test['Exterior1st']
del test['Exterior2nd']
del test["Condition2"]
del test['MSSubClass']
del test["Utilities"]

## Variable Exploration

In [3]:
#impute some missing values
columns = ["PoolQC", "MiscFeature", "Alley", "Fence", "FireplaceQu", 
           "GarageType", "GarageCond", "GarageFinish", "GarageQual", 
           "BsmtExposure", "BsmtFinType2", "BsmtFinType1", "BsmtCond", "BsmtQual"]

for col in columns:
    test[col] = test[col].fillna("NA")
    
test["LotFrontage"] = test["LotFrontage"].fillna(0)

In [4]:
test_con_cols = ['LotFrontage', 'LotArea',  'MasVnrArea','BsmtFinSF1', 'BsmtFinSF2', 'BsmtUnfSF','TotalBsmtSF',
                  '1stFlrSF', '2ndFlrSF','LowQualFinSF','GrLivArea','GarageArea','WoodDeckSF', 'OpenPorchSF',
                  'EnclosedPorch','3SsnPorch','ScreenPorch','PoolArea','MiscVal']

test_counts_cols = ['BsmtFullBath','BsmtHalfBath', 'FullBath', 'HalfBath', 'BedroomAbvGr', 'MSZoning', 
                     'KitchenAbvGr', 'TotRmsAbvGrd',  'Fireplaces', 'GarageCars', "remodel_flag", "shed_flag",
                   "pool_flag"]

test_cat = ["EraBuilt", "SeasonSold", "Neighborhood", "HouseStyle",'Street',
             'OverallQual', 'OverallCond','Alley','LotShape','LandContour','MSZoning',
             'LotConfig','LandSlope','Condition1', 'BldgType','HouseStyle','RoofStyle',
             'RoofMatl', 'MasVnrType', 'ExterQual','ExterCond','Foundation','BsmtQual','BsmtCond',
             'BsmtExposure','BsmtFinType1','BsmtFinType2','Heating','HeatingQC','CentralAir','Electrical',
             'KitchenQual','Functional','FireplaceQu','GarageType','GarageFinish','GarageQual','GarageCond',
             'PavedDrive', 'Fence','SaleType','SaleCondition']

drop_cols = ["YearBuilt", "MoSold", 'YearRemodAdd', "MiscFeature", "PoolQC"]

## Categorical Variables

In [5]:
test["Heating"] = np.where(test["Heating"].str.contains("Gas"), "Gas", "Other")
test["Electrical"] = np.where(test["Electrical"] == "SBrkr", "SBrkr", "Other")
test["GarageQual"] = np.where(test["GarageQual"].isin(["TA", "Gd", "Ex"]), "tpyical_good", "none_other")
test["pool_flag"] = np.where(test["PoolQC"] == "NA", 0, 1)
test["shed_flag"] = np.where(test["MiscFeature"] == "Shed", 1, 0)

In [6]:
test["SaleType"] = np.where(~test["SaleType"].isin(["WD", "New"]), "Other", test["SaleType"])

In [7]:
test["Condition1"] = np.where(test["Condition1"].isin(["RRAn", "RRAe", "RRNn", "RRNe"]), "rr_adj",
                           np.where(test["Condition1"].isin(["Artery", "Feedr"]), "road_adj",
                              np.where(test["Condition1"].isin(["PosA", "PosN"]), "pos_adj", 
                                       "norm"
                           )))

In [8]:
test["EraBuilt"] = np.where(test["YearBuilt"] < 1950, "pre_1950", 
                          np.where(test["YearBuilt"] < 1960, "1950s", 
                             np.where(test["YearBuilt"] < 1970, "1960s",
                                np.where(test["YearBuilt"] < 1980, "1970s", 
                                   np.where(test["YearBuilt"] < 1990, "1980s", 
                                      np.where(test["YearBuilt"] < 2000, "1990s", 
                                         "2000s"))))))

In [9]:
test["SeasonSold"] = np.where(test["MoSold"].isin([12, 1, 2]), "Winter", 
                           np.where(test["MoSold"].isin([3, 4, 5]), "Spring", 
                              np.where(test["MoSold"].isin([6, 7, 8]), "Summer",
                                       "Fall")))

In [10]:
test["remodel_flag"] = np.where(test["YearBuilt"] != test['YearRemodAdd'], 1, 0)

In [11]:
test['MSZoning'] = np.where(test['MSZoning'].str.contains("R"), "res", "non_res")
test["RoofMatl"] = np.where(test["RoofMatl"] == "CompShg", "CompShg", "other")

In [12]:
highend_hoods = ["NoRidge", "NridgeHt", "StoneBr", "Timber", "Veenker", "Somerst", 
                 "ClearCr", "Crawfor"]
midend_hoods = ["CollgCr", "Blmngtn", "Gilbert", "NWAmes", "SawyerW", "Mitchel", 
                "NAMes", "NPkVill"]
lowend_hoods = ["SWISU", "Blueste", "Sawyer", "OldTown", "Edwards","BrkSide", "BrDale", 
                "IDOTRR", "MeadowV"]
test["Neighborhood"] = np.where(test["Neighborhood"].isin(highend_hoods), "highend_hood", 
                             np.where(test["Neighborhood"].isin(midend_hoods), "midend_hood", 
                                  "lowend_hood"))

In [13]:
test["HouseStyle"] = np.where(test["HouseStyle"].isin(["1Story", "1.5Fin", "1.5Unf"]), "1Story", 
                             np.where(test["HouseStyle"].isin(["2Story", "2.5Fin", "2.5Unf"]), "2Story",
                                     "Split"))

In [14]:
test['OverallQual'] = np.where(test['OverallQual'] <= 5, "bad_quality", 
                            np.where(test['OverallQual'] <= 7, "med_quality", 
                                     "high_quality"))

In [15]:
test['OverallCond'] = np.where(test['OverallCond'] <= 4, "bad_cond", 
                            np.where(test['OverallCond'] == 5, "med_cond", 
                                     "great_cond"))

## Scaling

In [16]:
from sklearn import preprocessing

In [17]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
test[test_con_cols] = scaler.fit_transform(test[test_con_cols].fillna(0))

## Data For Model

In [18]:
# Get Dummy Variables
test_model = pd.get_dummies(test, columns = test_cat)
# Drop Duplicates
test_model = test_model.drop(test_cat, axis= 1).drop(drop_cols, axis = 1)

In [19]:
test_model.to_csv("test_model.csv")

In [20]:
test_model.shape

(1459, 211)